In [5]:
from web3 import Web3
import json
web3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
assert web3.isConnected()
SELECT_PROJECT = 0
SELECT_COMMIT = 0
COMMENT_MESSAGE = "good"

In [6]:
with open ("contract/abi.json") as f:
    contract_abi = f.read()
with open("config.json") as f:
    config = json.load(f)
contract_address = config["contract_address"]
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

In [7]:
private_key = "0x22aabb811efca4e6f4748bd18a46b502fa85549df9fa07da649c0a148d7d5530"
account = web3.eth.account.from_key(private_key)
web3.eth.default_account = account.address
print("address : %s" % account.address)
nonce = web3.eth.getTransactionCount(account.address)

address : 0xa508dD875f10C33C52a8abb20E16fc68E981F186


In [8]:
projects = contract.functions.get_all_projects_name().call()
if (len(projects) < SELECT_PROJECT):
    print ("Index out of bound")
commits = contract.functions.get_all_commit_logs(SELECT_PROJECT).call()
if (len(commits) < SELECT_COMMIT):
    print ("Index out of bound")

In [9]:
tx_params = {
    'nonce': nonce,
    'gasPrice': web3.eth.gas_price,
    'gas': 5000000, 
}

In [10]:
new_comment_tx = contract.functions.new_comment(SELECT_PROJECT,SELECT_COMMIT,COMMENT_MESSAGE).buildTransaction(tx_params)
signed_tx = web3.eth.account.sign_transaction(new_comment_tx, private_key)
try:
    tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print ("tx_receipt : %s" % tx_receipt)
except Exception as e:
    print (str(e).split("\'")[3])

tx_receipt : AttributeDict({'transactionHash': HexBytes('0x3aa65605ae4c342e23352a476c13ce88567cd48b205368798127f9c8186b3934'), 'transactionIndex': 0, 'blockHash': HexBytes('0xc9cf31804529f78b3b422c0efeeb20314df258c6346b43d30f2a57714cc27075'), 'blockNumber': 126, 'from': '0xa508dD875f10C33C52a8abb20E16fc68E981F186', 'to': '0x1062B8605e260A4CB5ADEbB5056C631554C4F289', 'gasUsed': 116506, 'cumulativeGasUsed': 116506, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [11]:
comments = contract.functions.get_comments(SELECT_PROJECT,SELECT_COMMIT).call()
for idx,c in enumerate(comments):
    print("%d. %s" % (idx+1,c))

1. ('yihuai', 'good')
